In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import torchvision.transforms as transforms
import torch.utils.data as dataloader

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
# hyperparams

num_epochs=5
lr=0.001
batch_size=4



In [4]:
transformation=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))])

train_dataset=torchvision.datasets.CIFAR10(root="./data",train=True, download=True,transform=transformation)
test_dataset=torchvision.datasets.CIFAR10(root="./data",train=False, download=True,transform=transformation)



In [5]:
train_data=dataloader.DataLoader(train_dataset,batch_size=4,shuffle=False)
test_data=dataloader.DataLoader(test_dataset,batch_size=4,shuffle=False)

In [6]:
classes=('plane','car','bird','cat','deer','dog','frog','horse','ship','truck')

In [7]:
class ConvNet(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1=nn.Conv2d(in_channels=3,out_channels=6,kernel_size=5)
    self.pool=nn.MaxPool2d(kernel_size=2, stride=2)
    self.conv2=nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
    # flatten 3d tensor to 1d tensor that is why 16*5*5
    self.fc1=nn.Linear(in_features=16*5*5,out_features=120)
    self.fc2=nn.Linear(in_features=120,out_features=84)
    self.fc3=nn.Linear(in_features=84,out_features=10)


    pass

  def forward(self,x):
    # -> 3,32,32
    x=self.conv1(x) # 6, 28, 28
    x=F.relu(x) # 6, 28, 28
    x=self.pool(x) # 6, 14, 14
    x=self.conv2(x) # 16 , 10, 10
    x=F.relu(x) # 16, 10 , 10
    x=self.pool(x) # 16, 5, 5
    # first flatten
    x=x.view(-1,16*5*5) # pytorch defines right size -1 is batch size #400
    x=self.fc1(x) # 120
    x=F.relu(x) # 120
    x=self.fc2(x) #84
    x=F.relu(x) # 84
    x=self.fc3(x) #10
    return x



In [8]:
model =ConvNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(model.parameters(),lr=lr)
n_total_steps = len(train_dataset)
for epoch in range(num_epochs):
  for i, (images,labels) in enumerate(train_data):

    images=images.to(device)
    labels=labels.to(device)

    #fwd pass

    outputs=model(images)
    loss=criterion(outputs,labels)

    # bwd pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i+1)%2000==0:
      print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}, Loss: {loss.item():.4f}]")

  print("Finished Training.")


Epoch [1/5], Step [2000/50000, Loss: 2.3222]
Epoch [1/5], Step [4000/50000, Loss: 2.3422]
Epoch [1/5], Step [6000/50000, Loss: 2.3063]
Epoch [1/5], Step [8000/50000, Loss: 2.2918]
Epoch [1/5], Step [10000/50000, Loss: 2.1214]
Epoch [1/5], Step [12000/50000, Loss: 2.2835]
Finished Training.
Epoch [2/5], Step [2000/50000, Loss: 2.6860]
Epoch [2/5], Step [4000/50000, Loss: 2.2951]
Epoch [2/5], Step [6000/50000, Loss: 1.5184]
Epoch [2/5], Step [8000/50000, Loss: 1.7361]
Epoch [2/5], Step [10000/50000, Loss: 2.4663]
Epoch [2/5], Step [12000/50000, Loss: 1.5189]
Finished Training.
Epoch [3/5], Step [2000/50000, Loss: 2.5871]
Epoch [3/5], Step [4000/50000, Loss: 2.1803]
Epoch [3/5], Step [6000/50000, Loss: 1.6547]
Epoch [3/5], Step [8000/50000, Loss: 1.6294]
Epoch [3/5], Step [10000/50000, Loss: 2.6447]
Epoch [3/5], Step [12000/50000, Loss: 1.1942]
Finished Training.
Epoch [4/5], Step [2000/50000, Loss: 2.3736]
Epoch [4/5], Step [4000/50000, Loss: 2.1694]
Epoch [4/5], Step [6000/50000, Loss: 

In [12]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_data:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc} %')

Accuracy of the network: 48.9 %
Accuracy of plane: 43.9 %
Accuracy of car: 77.3 %
Accuracy of bird: 28.7 %
Accuracy of cat: 32.3 %
Accuracy of deer: 32.3 %
Accuracy of dog: 37.3 %
Accuracy of frog: 62.3 %
Accuracy of horse: 68.7 %
Accuracy of ship: 63.5 %
Accuracy of truck: 42.7 %
